# function random

In [1]:
import random

def gen_lo_up(): # generate HSV at random
    # 定義HSV_mask 的上下限值
    seed = random.getstate()
    random.setstate(seed)

    lower = [0]*3
    upper = [0]*3
    lower[0] = random.randint(0,180)
    upper[0] = random.randint(lower[0],180)

    for i in range(1,3):
        lower[i] = random.randint(0,255)
        upper[i] = random.randint(lower[i],255)
    
    lo = np.array([lower[0],lower[1],lower[2]])
    up = np.array([upper[0],upper[1],upper[2]])

    return lo, up

    # lower_blue = np.array([lower[0],lower[1],lower[2]])
    # upper_blue = np.array([upper[0],upper[1],upper[2]])

# find useable lower & upper value

In [2]:
def find_lo_up(img, lower_blue, upper_blue):
    # # load window 
    # win_name = "result"
    # cv.namedWindow(win_name, cv.WINDOW_KEEPRATIO)
    # cv.resizeWindow(win_name, 1280, 960)

    # copy_img_1 = img.copy()
    # copy_img_2 = img.copy()
    # clone= img.copy()

    # select lower & upper value
    while(True):
        
        # 先將原影像由 BGR 色域轉成 HSV 色域並顯示
        hsv_img = cv.cvtColor(img, cv.COLOR_BGR2HSV)

        # 透過 inRange() 函式形成白色遮罩，遮罩區域為原始影像區域
        hsv_bin_mask = cv.inRange(hsv_img, lower_blue, upper_blue)

        # 套用 hsv_bin_mask 到原圖上進行過濾
        hsv_mask = cv.bitwise_and(img, img, None, mask=hsv_bin_mask)

        # 將我們的 HSV Mask 套用OpenCV內建的Contours輪廓函式，在我們目標物體畫出邊框

        # 找輪廓 the one greater than expect
        (cnts, _) = cv.findContours(hsv_bin_mask.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)  # 回傳輪廓list

        if(cnts == ()): 
            # print("No cnts") # No cnts
            return -1 # quit loop

        else:   # detect cnts have content
            max_area = 0
            (x, y, w, h) = cv.boundingRect(cnts[max_area])  # 輪廓矩形
            for i in range(len(cnts)): # find max rect
                (_x, _y, _w, _h) = cv.boundingRect(cnts[i])  # 矩形 rect
                if ( ( _w * _h ) > ( w * h ) ):
                    max_area = i
                    (x, y, w, h) = cv.boundingRect(cnts[max_area])

            # find max area %
            ratio = (h * w) / (img.shape[0] * img.shape[1]) #hieght,width
            # print("ratio : {:.4f}".format(ratio))
            
            return lower_blue ,upper_blue, ratio # find interval, process next

    #################################
    # cv.drawContours(clone, cnts, -1, (0, 255, 0), 2) # 畫輪廓

    # cv.rectangle(copy_img_1, (x, y), (x + w, y + h), (0, 0, 255), 1)  # 畫矩形

    # BoundingBox = cv.minAreaRect(cnts[max_area])  # 最小外接矩形的中心（x，y），（寬度，高度），旋轉角度）
    # BoundingBox = np.int0(cv.boxPoints(BoundingBox))  # int0 會省略小數點後方的數字
    # cv.drawContours(copy_img_2, [BoundingBox], -1, (0, 255, 0), 1)

    # # 顯示影像
    # image_1 = cv.hconcat([hsv_mask, clone])
    # image_2 = cv.hconcat([copy_img_1, copy_img_2])
    # image   = cv.vconcat([image_1, image_2])
    # cv.imshow(win_name, image)

    # # 按下任意鍵關閉視窗
    # cv.waitKey(0)
    # cv.destroyAllWindows()


# Result: (must have useable lower & upper first)

In [13]:
def result_perview():
    # 讀取影像
    img = cv.imread(img_name)

    # 先將原影像由 BGR 色域轉成 HSV 色域並顯示
    hsv_img = cv.cvtColor(img, cv.COLOR_BGR2HSV)

    # 透過 inRange() 函式形成白色遮罩，遮罩區域為原始影像區域
    hsv_bin_mask = cv.inRange(hsv_img, lower_blue, upper_blue)

    # 套用 hsv_bin_mask 到原圖上進行過濾
    hsv_mask = cv.bitwise_and(img, img, None, mask=hsv_bin_mask)

    # 將我們的 HSV Mask 套用OpenCV內建的Contours輪廓函式，在我們目標物體畫出邊框

    # 找輪廓
    (cnts, _) = cv.findContours(hsv_bin_mask.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)  # 回傳輪廓list

    # load window 
    win_name = "result"
    cv.namedWindow(win_name, cv.WINDOW_KEEPRATIO)
    cv.resizeWindow(win_name, 1280, 960)

    copy_img_1=img.copy()
    copy_img_2 = img.copy()
    clone= img.copy()

    cv.drawContours(clone, cnts, -1, (0, 255, 0), 2)

    for cnt in cnts:
        (x, y, w, h) = cv.boundingRect(cnt)  # 矩形 rect
        if(w*h> 10):
            cv.rectangle(copy_img_1, (x, y), (x + w, y + h), (0, 0, 255), 1)  # 畫矩形
            
            BoundingBox = cv.minAreaRect(cnt)  # 最小外接矩形的中心（x，y），（寬度，高度），旋轉角度）
            BoundingBox = np.int0(cv.boxPoints(BoundingBox))  # int0 會省略小數點後方的數字
            cv.drawContours(copy_img_2, [BoundingBox], -1, (0, 255, 0), 1)

    # 顯示影像
    image_1 = cv.hconcat([hsv_mask, clone])
    image_2 = cv.hconcat([copy_img_1, copy_img_2])
    image   = cv.vconcat([image_1, image_2])
    cv.imshow(win_name, image)
    # cv.imshow('Contour', clone)  # 輪廓
    # cv.imshow('Bounding Rectangle', copy_img_1)  # 矩形
    # cv.imshow('Minimum Area Rectangle', copy_img_2)  # 最小外接矩形

    # 按下任意鍵關閉視窗
    cv.waitKey(0)
    cv.destroyAllWindows()


# run main

In [15]:
import numpy as np
import cv2 as cv

# 讀取影像
img_name = 'contrasthigh.jpg'
img = cv.imread(img_name)
img = cv.resize(img, (640, 480), interpolation=cv.INTER_AREA)
cv.imshow("original", img)
cv.waitKey(0)
cv.destroyAllWindows()

# set img cut01- with object
img_cut01 = img[70:142, 320:400]
# img_cut01 = img[270:401, 400:515] # y_up, y_down, x_up , x_down
cv.imshow("cut01", img_cut01)

# set img cut02- without object
img_cut02 = img[150:200, 0:640]
# img_cut02 = img[30:337, 215:393] # y_up, y_down, x_up , x_down
cv.imshow("cut02", img_cut02)

cv.waitKey(0)
cv.destroyAllWindows()

while(True):
    lower_blue ,upper_blue = gen_lo_up() # generate interval at random
    
    # 1.flag == -1 -> haven't cnts
    # 2.flag != -1 -> have    cnts

    # cut01- with object
    flag = find_lo_up(img_cut01, lower_blue, upper_blue) # detect useable or not
    if( flag != -1 ): 
        lower_blue ,upper_blue, ratio_01 = flag
        if(ratio_01 > 0.80):
            pass
        else:
            continue
    else:
        continue

    # cut02- without object
    flag = find_lo_up(img_cut02, lower_blue, upper_blue) # detect useable or not
    if( flag != -1 ): 
        # continue
        lower_blue ,upper_blue, ratio_02 = flag
        if(ratio_02 < 0.0001):
            pass
        else:
            continue
    else: # flag == -1
        ratio_02 = 0
        pass
    
    # compare both
    precise_rate = ratio_01 * (1-ratio_02)
    if(precise_rate > 0.80):
        print("find interval : " ,lower_blue ,upper_blue)
        print("ratio fron cut02 : " ,ratio_01)
        print("ratio fron cut03 : " ,ratio_02)
        print("precise_rate : " ,precise_rate)
        break
    else:
        continue
    
result_perview()
# lower_blue = np.array([30,70,0])
# upper_blue = np.array([55,255,185])
# print(lower_blue, upper_blue)

find interval :  [ 92 179  14] [148 255 255]
ratio fron cut02 :  0.8333333333333334
ratio fron cut03 :  0
precise_rate :  0.8333333333333334
